In [1]:
import pandas as pd
import numpy as np
import re
from Bio import SeqIO

# **Reading of .fasta files**
Here we define functions that will allow us to read the .fasta files of our positive and negative dataset results from CD-HIT and MMSeqs2 and create their pandas dataframes. 

## CD-HIT Datasets

### CD-HIT Positive .fasta

In [2]:
## Reads the CD-HIT .fasta file output of the positive_dataset.fasta and converts into a pandas dataframe (clustered_positive_cdhit.fasta)
def fasta2df_positive_cdhit(fasta_file):
    """
    Reads the CD-HIT .fasta file output of the positive dataset and converts it into a pandas DataFrame.

    Parameters:
    fasta_file (str): Path to the CD-HIT clustered .fasta file of the positive dataset.

    Returns:
    pd.DataFrame: DataFrame with columns:
        - 'ID': Accession IDs of the sequences.
        - 'Sequence': Corresponding protein sequences.
        - 'Organism': Organism information extracted from the sequence description.
        - 'Family': Family information extracted from the sequence description.
        - 'Type': Type information extracted from the sequence description.
        - 'Size': Length of the protein sequences or value extracted from metadata.
        - 'Protein Acr': Protein acronym extracted from the sequence description.
    """
    # Read the FASTA file
    fasta_sequences = SeqIO.parse(open(fasta_file), 'fasta')

    # Create a list of dictionaries containing the sequences and their metadata
    data = []
    for fasta in fasta_sequences:
        # Extract metadata from the description line of the FASTA record
        description = fasta.description
        identifier = fasta.id                                               # 'ID' column (Accession ID)
        sequence = str(fasta.seq)                                           # 'Sequence' column

        # Use regular expressions to extract metadata
        matches = re.findall(r'(\w+)=([^=]+?)\s*(?=\w+=|$)', description)  # Updated regular expression
        metadata = dict(matches)

        # Append the data to the list as a dictionary
        data.append({'ID': identifier, 
                     'Sequence': sequence, 
                     'Organism': metadata.get('Organism'), 
                     'Family': metadata.get('Family'), 
                     'Type': metadata.get('Type'), 
                     'Size': int(metadata.get('Size', 0)), 
                     'Protein Acr': metadata.get('ProteinAcr')})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(data)
    return df

In [3]:
# For CD-HIT result of positive_dataset.fasta:
df_positive_cdhit = fasta2df_positive_cdhit("CDHIT_MMSeqs2_Files/clustered_positive_cdhit.fasta")
df_positive_cdhit   # CD-HIT positive_dataset.fasta Output

,ID,Sequence,Organism,Family,Type,Size,Protein Acr
0,RGB60049.1,MSIYTDMIPAILLVNDPQDSLSGAPIENYVKVSNINVAIYKNDSFK...,Absiella sp.,AcrIIA8,II-A,105,Yes
1,WP_103240931.1,MSCPFQAMEGGNGMERKMALREFCGRYRKGDFKGTERAVQIEAGWY...,Acetatifactor muris,AcrIIA11,II-A,195,Yes
2,WP_086652143.1,MELIHTSDEVIKKIHKDGTFDTFLFFSASKYLAGSVASRKHYTYKI...,Acetobacter cibinongensis,AcrIF11,I-F,179,Yes
3,WP_012242545.1,MEKQQLLKDLIQAFNSGSFESSDVKVQIKAGWYDWFCKDSSLKNKT...,Acholeplasma laidlawii,AcrIIA11,II-A,144,Yes
4,WP_062681378.1,MQLFHTSPSEISTITSTGRFGSFLFFSARAYTMTAGEALVYSLEID...,Achromobacter denitrificans,AcrIF11,I-F,150,Yes
...,...,...,...,...,...,...,...
1117,CNL87740.1,MNFGQALQALKAGHKVARIGWNGKGMFPILISGTKDVEPCEGTPYA...,Yersinia pseudotuberculosis,AcrIIA7,II-A,146,Yes
1118,WP_050090803.1,MKLFHGSYSKTAPVIKVGAYALGSSDNIFDGLFASADIEIASSHGN...,Yersinia pseudotuberculosis,AcrIF11,I-F,162,Yes
1119,WP_071704514.1,MNFGEALEAVKSGKKIARSGWNGAAQFVIKAGGYTVSEARPGSDYA...,Yersinia ruckeri,AcrIIA7,II-A,88,Yes
1120,CFQ72446.1,MKLFHGSYSSTAPVMQIGQFTQVNGSENLYDGIFASDSMDSASSHG...,Yersinia similis,AcrIF11,I-F,141,Yes


### CD-HIT Negative .fasta

In [4]:
## Reads the CD-HIT .fasta file output of the negative_dataset.fasta and converts into a pandas dataframe (clustered_negative_cdhit.fasta)
def fasta2df_negative_cdhit(fasta_file):
    """
    Reads the CD-HIT .fasta file output of the negative dataset and converts it into a pandas DataFrame.

    Parameters:
    fasta_file (str): Path to the CD-HIT clustered .fasta file of the negative dataset.

    Returns:
    pd.DataFrame: DataFrame with columns:
        - 'ID': Accession IDs of the sequences.
        - 'Sequence': Corresponding protein sequences.
        - 'Organism': Organism information extracted from the sequence description.
        - 'Structure': Description of the protein structure.
        - 'Size': Length of the protein sequences.
        - 'Protein Acr': A column with the value 'No' for all entries.
    """
    # Read the FASTA file and extract required information for each record
    data = [(fasta.id,
             str(fasta.seq),
             re.search(r'\[(.*?)\]', fasta.description).group(1),
             re.sub(r'\[.*?\]', '', fasta.description).strip().replace(fasta.id, '').strip())
            for fasta in SeqIO.parse(open(fasta_file), 'fasta')]

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=['ID', 'Sequence', 'Organism', 'Structure'])

    # Add new columns 'Size' and 'Protein Acr'
    df['Size'] = df['Sequence'].apply(len)
    df['Protein Acr'] = 'No'

    return df

In [5]:
# For CD-HIT result of negative_dataset.fasta:
df_negative_cdhit = fasta2df_negative_cdhit('CDHIT_MMSeqs2_Files/clustered_negative_cdhit.fasta')
df_negative_cdhit   # CD-HIT negative_dataset.fasta Output 

,ID,Sequence,Organism,Structure,Size,Protein Acr
0,WP_132722810.1,MTNNTDTFLHYREGKTQSQRFLKNLDPENLKLNDLDVADWLLFAFN...,Tenacibaculum sp. M341,phage baseplate protein,1042,No
1,WP_130915135.1,MKKTDTFSHYREGKSQMQRFLAELDPGNLELHDFDLFDWLLFANNF...,Chryseobacterium gleum,phage baseplate protein,1027,No
2,AYZ12950.1,MKKTDTFSHYREGKSQMQRFLAELDPGNLELHDFDLFDWLLFANNF...,Chryseobacterium arthrosphaerae,phage baseplate protein,1016,No
3,WP_124535473.1,MKKTDTFSHYREGKSQMQRFLAELDPSNLELHDFDLFDWLLFANNF...,Chryseobacterium sp. KBW03,phage baseplate protein,1017,No
4,WP_123887321.1,MKKTDTFSHYREGKSQMQRFLAELDPGNLELHDFDLFDWLLFANNF...,Chryseobacterium indoltheticum,phage baseplate protein,1016,No
...,...,...,...,...,...,...
57374,AXY81356.1,MNAHTPFDAKQDWTNPYCQNSSNDPMVDALLGNAYHVVRTVYCNLG...,Escherichia phage PD38,tail fiber protein,905,No
57375,AGC31500.1,MNAHTPFDAKKDWSNPYCQNSSNDPMVDALLGNAYHVVRTVYCNLG...,Escherichia phage EC1-UPM,tail fiber protein,928,No
57376,AEL79676.1,MNAHTPFDANQDWSNPYCQNKSNDPMVDALLGNAYHVVRTVYCNLG...,Escherichia phage vB_EcoP_G7C,tail fiber protein,1063,No
57377,EHJ79759.1,MDNEFYTLLTDRGMAKIASALADKKQIHLQKMAVGDGGGQYYEPTA...,Salmonella enterica subsp. enterica serovar Ba...,Phage tail fiber protein,476,No


## MMSeqs2 Datasets

### MMSeqs2 Positive .fasta

In [6]:
## Reads the MMSeqs2 .fasta output of the positive dataset and then fills in the missing column values by extracting them from the original positive_dataset.fasta file via Accession IDs.
def fasta2df_positive_mmseqs(original_fasta, mmseqs2_fasta):
    """
    Reads .fasta files from the MMSeqs2 positive dataset output and its original positive dataset,
    then merges them together by filling in missing values from the original dataset using Accession IDs.

    Parameters:
    original_fasta (str): Path to the original .fasta file of the positive dataset.
    mmseqs2_fasta (str): Path to the MMSeqs2 .fasta output of the positive dataset.

    Returns:
    pd.DataFrame: DataFrame with columns:
        - 'ID': Accession IDs of the sequences.
        - 'Sequence': Corresponding protein sequences.
        - 'Organism': Organism information extracted from the sequence description.
        - 'Family': Family information extracted from the sequence description.
        - 'Type': Type information extracted from the sequence description.
        - 'Size': Length of the protein sequences or value extracted from metadata.
        - 'Protein Acr': Protein acronym extracted from the sequence description.
    """
    # Read the original fasta file to create a dictionary of accession ID and associated values
    original_data = {}
    for record in SeqIO.parse(original_fasta, 'fasta'):
        description = record.description
        accession_id = record.id
        # Use regular expressions to extract metadata
        matches = re.findall(r'(\w+)=([^=]+?)\s*(?=\w+=|$)', description)
        metadata = dict(matches)
        original_data[accession_id] = metadata

    # Read the mmseqs2 fasta file and try to assign missing values from the original fasta
    filled_data = []
    for record in SeqIO.parse(mmseqs2_fasta, 'fasta'):
        accession_id = record.id
        sequence = str(record.seq)
        metadata = original_data.get(accession_id, {})
        # Append the data to the list as a dictionary
        filled_data.append({'ID': accession_id, 
                            'Sequence': sequence, 
                            'Organism': metadata.get('Organism'), 
                            'Family': metadata.get('Family'), 
                            'Type': metadata.get('Type'), 
                            'Size': int(metadata.get('Size', 0)), 
                            'Protein Acr': metadata.get('ProteinAcr')})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(filled_data)

    # Sort the DataFrame alphabetically based on the "Organism" column
    df = df.sort_values(by='Organism')
    df = df.reset_index(drop=True) # Reset index
    
    return df

In [7]:
# For MMSeqs2 result of positive_dataset.fasta:
df_positive_mmseqs = fasta2df_positive_mmseqs('positive_dataset.fasta', 'CDHIT_MMSeqs2_Files/positive_output_mmseqs.fasta')
df_positive_mmseqs   # MMSeqs2 positive_dataset.fasta Output

,ID,Sequence,Organism,Family,Type,Size,Protein Acr
0,RGB60049.1,MSIYTDMIPAILLVNDPQDSLSGAPIENYVKVSNINVAIYKNDSFK...,Absiella sp.,AcrIIA8,II-A,105,Yes
1,WP_103240931.1,MSCPFQAMEGGNGMERKMALREFCGRYRKGDFKGTERAVQIEAGWY...,Acetatifactor muris,AcrIIA11,II-A,195,Yes
2,WP_086652143.1,MELIHTSDEVIKKIHKDGTFDTFLFFSASKYLAGSVASRKHYTYKI...,Acetobacter cibinongensis,AcrIF11,I-F,179,Yes
3,WP_012242545.1,MEKQQLLKDLIQAFNSGSFESSDVKVQIKAGWYDWFCKDSSLKNKT...,Acholeplasma laidlawii,AcrIIA11,II-A,144,Yes
4,WP_062681378.1,MQLFHTSPSEISTITSTGRFGSFLFFSARAYTMTAGEALVYSLEID...,Achromobacter denitrificans,AcrIF11,I-F,150,Yes
...,...,...,...,...,...,...,...
1101,WP_011192267.1,MNFGQALQALKAGYKVARIGWNGKGMFLILISGTKDVEPCEGTPYA...,Yersinia pseudotuberculosis,AcrIIA7,II-A,146,Yes
1102,AIN13836.1,MDSASSHGCGFVYTYEINDEKIASSRDLDNRTEEVYAFLRAELNID...,Yersinia pseudotuberculosis,AcrIF11,I-F,124,Yes
1103,WP_071704514.1,MNFGEALEAVKSGKKIARSGWNGAAQFVIKAGGYTVSEARPGSDYA...,Yersinia ruckeri,AcrIIA7,II-A,88,Yes
1104,CFQ72446.1,MKLFHGSYSSTAPVMQIGQFTQVNGSENLYDGIFASDSMDSASSHG...,Yersinia similis,AcrIF11,I-F,141,Yes


### MMSeqs2 Negative .fasta

In [8]:
## Reads the MMSeqs2 .fasta file of the negative dataset (negative_output_mmseqs.fasta)
def fasta2df_negative_mmseqs(mmseqs2_fasta):
    """
    Reads .fasta files from the MMSeqs2 negative dataset output and converts it to a pandas DataFrame.

    Parameters:
    mmseqs2_fasta (str): Path to the MMSeqs2 negative dataset .fasta file.

    Returns:
    pd.DataFrame: DataFrame with two columns:
        - 'ID': Accession IDs of the sequences.
        - 'Sequence': Corresponding protein sequences.
    """
    # Read the mmseqs2 fasta file
    filled_data = []
    for record in SeqIO.parse(mmseqs2_fasta, 'fasta'):
        accession_id = record.id
        sequence = str(record.seq)
        # Append the data to the list as a dictionary
        filled_data.append({'ID': accession_id, 
                            'Sequence': sequence})

    # Create a DataFrame from the list of dictionaries
    df = pd.DataFrame(filled_data)

    return df

## Reads the original .fasta file of the negative dataset (negative_dataset.fasta)
def fasta2df_negative_original(original_fasta):
    """
    Reads .fasta files of the original negative dataset and converts it to a pandas DataFrame.

    Parameters:
    original_fasta (str): Path to the original negative dataset .fasta file.

    Returns:
    pd.DataFrame: DataFrame with columns:
        - 'ID': Accession IDs of the sequences.
        - 'Sequence': Corresponding protein sequences.
        - 'Organism': Organism information extracted from the sequence description.
        - 'Structure': Description of the protein structure.
        - 'Size': Length of the protein sequences.
        - 'Protein Acr': A column with the value 'No' for all entries.
    """
    # Read the FASTA file and extract required information for each record
    data = [(fasta.id,
             str(fasta.seq),
             re.search(r'\[(.*?)\]', fasta.description).group(1),
             re.sub(r'\[.*?\]', '', fasta.description).strip().replace(fasta.id, '').strip())
            for fasta in SeqIO.parse(open(original_fasta), 'fasta')]

    # Create a DataFrame from the extracted data
    df = pd.DataFrame(data, columns=['ID', 'Sequence', 'Organism', 'Structure'])

    # Add new columns 'Size' and 'Protein Acr'
    df['Size'] = df['Sequence'].apply(len)
    df['Protein Acr'] = 'No'

    return df

In [9]:
# For MMSeqs2 result of negative_dataset.fasta:
df_negative_original = fasta2df_negative_original('negative_dataset.fasta')
df_negative_mmseqs = fasta2df_negative_mmseqs('CDHIT_MMSeqs2_Files/negative_output_mmseqs.fasta')

df_negative_mmseqs = pd.merge(df_negative_mmseqs, df_negative_original, on=['ID', 'Sequence'], how='inner')
df_negative_mmseqs   # MMSeqs2 negative_dataset.fasta Output

,ID,Sequence,Organism,Structure,Size,Protein Acr
0,WP_132722810.1,MTNNTDTFLHYREGKTQSQRFLKNLDPENLKLNDLDVADWLLFAFN...,Tenacibaculum sp. M341,phage baseplate protein,1042,No
1,WP_123214754.1,MADCNDILDILKRDGTGQEQRYTEALRTDYFRLQDLDIADWMLFAY...,Sinomicrobium pectinilyticum,phage baseplate protein,1051,No
2,ARW58754.1,MGIYVKLINVPVSERIYCDLNAWIQLEPRDNVQNMDSIVQKILMIV...,Erwinia phage vB_EamM_Y3,baseplate protein,139,No
3,WP_096978906.1,MERDLYTPTGAQTSEAQLLEYTFEMLLSGCFFIELAKVVAVRGKAP...,Escherichia coli,phage baseplate protein,230,No
4,WP_050336321.1,MNNPYSQSQNQSNDSDAFASSFNKLLNSNYFIRLATVTAVRGTAPN...,Yersinia enterocolitica,phage baseplate protein,236,No
...,...,...,...,...,...,...
58281,WP_011840497.1,MDKEIAAILERVVKDVARIDNELSKKAEAAFAEVKNMGNLSTETKA...,Rhodobacter sphaeroides,phage major capsid protein,403,No
58282,AXC34520.1,MKTRNKLTEALLGRGGFAELFAPKPTNKEEIKNEEKLLKADQMVEM...,Vibrio phage YC,portal protein,566,No
58283,YP_009222143.1,MATWKSRRKQKHEAKQKALLEKASSEAPAPRLRLGEIGTTGLAAIQ...,Vibrio phage qdvp001,portal protein,554,No
58284,WP_057903243.1,MVLEVNAPLLDDRFIEYARISNNGTFFYPAELEISTGDLLNLVDYH...,Lactobacillus bifermentans,phage portal protein,499,No


### Now that we have all our .fasta files in pandas dataframe format, we may proceed with the Propythia package
__________

# **Propythia** - Generate Descriptors

Dataframes:

- CD-HIT Dataframes:
    - df_positive_cdhit (Positive Dataset CD-HIT)
    - df_negative_cdhit (Negative Dataset CD-HIT)

- MMSeqs2 Dataframes:
    - df_positive_mmseqs (Positive Dataset MMSeqs2)
    - df_negative_mmseqs (Negative Dataset MMSeqs2)

In [10]:
from propythia.protein.sequence import ReadSequence
from propythia.protein.descriptors import ProteinDescritors
from sklearn.preprocessing import LabelEncoder

## - CD-HIT Dataframes
- Application of Propythia on both positive and negative CD-HIT results;
- Random selection of rows from negative dataset;
- Creation of CD-HIT dataframe.

### df_positive_cdhit

In [11]:
read_seqs = ReadSequence()
res = read_seqs.par_preprocessing(dataset=df_positive_cdhit, col='Sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
res

,ID,Sequence,Organism,Family,Type,Size,Protein Acr
0,RGB60049.1,MSIYTDMIPAILLVNDPQDSLSGAPIENYVKVSNINVAIYKNDSFK...,Absiella sp.,AcrIIA8,II-A,105,Yes
1,WP_103240931.1,MSCPFQAMEGGNGMERKMALREFCGRYRKGDFKGTERAVQIEAGWY...,Acetatifactor muris,AcrIIA11,II-A,195,Yes
2,WP_086652143.1,MELIHTSDEVIKKIHKDGTFDTFLFFSASKYLAGSVASRKHYTYKI...,Acetobacter cibinongensis,AcrIF11,I-F,179,Yes
3,WP_012242545.1,MEKQQLLKDLIQAFNSGSFESSDVKVQIKAGWYDWFCKDSSLKNKT...,Acholeplasma laidlawii,AcrIIA11,II-A,144,Yes
4,WP_062681378.1,MQLFHTSPSEISTITSTGRFGSFLFFSARAYTMTAGEALVYSLEID...,Achromobacter denitrificans,AcrIF11,I-F,150,Yes
...,...,...,...,...,...,...,...
1117,CNL87740.1,MNFGQALQALKAGHKVARIGWNGKGMFPILISGTKDVEPCEGTPYA...,Yersinia pseudotuberculosis,AcrIIA7,II-A,146,Yes
1118,WP_050090803.1,MKLFHGSYSKTAPVIKVGAYALGSSDNIFDGLFASADIEIASSHGN...,Yersinia pseudotuberculosis,AcrIF11,I-F,162,Yes
1119,WP_071704514.1,MNFGEALEAVKSGKKIARSGWNGAAQFVIKAGGYTVSEARPGSDYA...,Yersinia ruckeri,AcrIIA7,II-A,88,Yes
1120,CFQ72446.1,MKLFHGSYSSTAPVMQIGQFTQVNGSENLYDGIFASDSMDSASSHG...,Yersinia similis,AcrIF11,I-F,141,Yes


In [12]:
descriptors_df = ProteinDescritors(dataset=df_positive_cdhit , col='Sequence')
df_positive_cdhit_descriptors = descriptors_df.get_all_physicochemical(n_jobs = -1)     # Obtain physicochemical features of our sequences
# df_positive_cdhit_descriptors = descriptors_df.get_all(n_jobs = -1) # Obtain all features of our sequences. Note: When running this code and then performing ML, the results of each model hinted towards Overfitting behaviour!
df_positive_cdhit_descriptors

,ID,Sequence,Organism,Family,Type,Size,Protein Acr,length,charge,chargedensity,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,RGB60049.1,MSIYTDMIPAILLVNDPQDSLSGAPIENYVKVSNINVAIYKNDSFK...,Absiella sp.,AcrIIA8,II-A,105,Yes,105.0,-2.901,-0.000240,...,4.980767,34.719048,0.266667,0.323810,0.390476,14440,14440,82.666667,1.931333,0.361905
1,WP_103240931.1,MSCPFQAMEGGNGMERKMALREFCGRYRKGDFKGTERAVQIEAGWY...,Acetatifactor muris,AcrIIA11,II-A,195,Yes,195.0,-17.844,-0.000784,...,4.608869,34.520000,0.343590,0.282051,0.307692,50420,50670,67.538462,2.295333,0.343590
2,WP_086652143.1,MELIHTSDEVIKKIHKDGTFDTFLFFSASKYLAGSVASRKHYTYKI...,Acetobacter cibinongensis,AcrIF11,I-F,179,Yes,179.0,-20.646,-0.001021,...,4.386572,54.459832,0.402235,0.217877,0.346369,25900,25900,81.899441,1.828212,0.391061
3,WP_012242545.1,MEKQQLLKDLIQAFNSGSFESSDVKVQIKAGWYDWFCKDSSLKNKT...,Acholeplasma laidlawii,AcrIIA11,II-A,144,Yes,144.0,-0.834,-0.000050,...,6.289712,33.052083,0.291667,0.291667,0.402778,41940,42065,82.430556,1.778750,0.319444
4,WP_062681378.1,MQLFHTSPSEISTITSTGRFGSFLFFSARAYTMTAGEALVYSLEID...,Achromobacter denitrificans,AcrIF11,I-F,150,Yes,150.0,-19.684,-0.001198,...,4.110619,34.327400,0.340000,0.286667,0.373333,11460,11460,98.333333,1.572533,0.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,CNL87740.1,MNFGQALQALKAGHKVARIGWNGKGMFPILISGTKDVEPCEGTPYA...,Yersinia pseudotuberculosis,AcrIIA7,II-A,146,Yes,146.0,-5.762,-0.000366,...,4.923246,27.177397,0.301370,0.294521,0.369863,19480,19480,94.794521,0.922123,0.397260
1118,WP_050090803.1,MKLFHGSYSKTAPVIKVGAYALGSSDNIFDGLFASADIEIASSHGN...,Yersinia pseudotuberculosis,AcrIF11,I-F,162,Yes,162.0,-19.388,-0.001103,...,4.306713,39.247531,0.327160,0.333333,0.327160,11460,11460,88.024691,1.628951,0.407407
1119,WP_071704514.1,MNFGEALEAVKSGKKIARSGWNGAAQFVIKAGGYTVSEARPGSDYA...,Yersinia ruckeri,AcrIIA7,II-A,88,Yes,88.0,0.198,0.000021,...,6.711741,27.969318,0.306818,0.340909,0.295455,19480,19480,72.159091,0.951818,0.363636
1120,CFQ72446.1,MKLFHGSYSSTAPVMQIGQFTQVNGSENLYDGIFASDSMDSASSHG...,Yersinia similis,AcrIF11,I-F,141,Yes,141.0,-15.925,-0.001013,...,4.096693,41.008511,0.283688,0.361702,0.333333,23950,24075,78.226950,1.891064,0.375887


### df_negative_cdhit

In [13]:
read_seqs = ReadSequence()
res = read_seqs.par_preprocessing(dataset=df_negative_cdhit, col='Sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
res

,ID,Sequence,Organism,Structure,Size,Protein Acr
0,WP_132722810.1,MTNNTDTFLHYREGKTQSQRFLKNLDPENLKLNDLDVADWLLFAFN...,Tenacibaculum sp. M341,phage baseplate protein,1042,No
1,WP_130915135.1,MKKTDTFSHYREGKSQMQRFLAELDPGNLELHDFDLFDWLLFANNF...,Chryseobacterium gleum,phage baseplate protein,1027,No
2,AYZ12950.1,MKKTDTFSHYREGKSQMQRFLAELDPGNLELHDFDLFDWLLFANNF...,Chryseobacterium arthrosphaerae,phage baseplate protein,1016,No
3,WP_124535473.1,MKKTDTFSHYREGKSQMQRFLAELDPSNLELHDFDLFDWLLFANNF...,Chryseobacterium sp. KBW03,phage baseplate protein,1017,No
4,WP_123887321.1,MKKTDTFSHYREGKSQMQRFLAELDPGNLELHDFDLFDWLLFANNF...,Chryseobacterium indoltheticum,phage baseplate protein,1016,No
...,...,...,...,...,...,...
57374,AXY81356.1,MNAHTPFDAKQDWTNPYCQNSSNDPMVDALLGNAYHVVRTVYCNLG...,Escherichia phage PD38,tail fiber protein,905,No
57375,AGC31500.1,MNAHTPFDAKKDWSNPYCQNSSNDPMVDALLGNAYHVVRTVYCNLG...,Escherichia phage EC1-UPM,tail fiber protein,928,No
57376,AEL79676.1,MNAHTPFDANQDWSNPYCQNKSNDPMVDALLGNAYHVVRTVYCNLG...,Escherichia phage vB_EcoP_G7C,tail fiber protein,1063,No
57377,EHJ79759.1,MDNEFYTLLTDRGMAKIASALADKKQIHLQKMAVGDGGGQYYEPTA...,Salmonella enterica subsp. enterica serovar Ba...,Phage tail fiber protein,476,No


##### Random selection of n rows (negative) = n rows (positive)

In [14]:
# Random selection of n rows through pandas sample() method (where n = number of rows in the positive counterpart)
# Note: Paramater 'random_state' of the sample() method ensures reproducibility of the random selection
df_negative_cdhit_randomsampled = df_negative_cdhit.sample(n=df_positive_cdhit_descriptors.shape[0], random_state=1) 
df_negative_cdhit_randomsampled

,ID,Sequence,Organism,Structure,Size,Protein Acr
10103,OCQ54497.1,MKYFAILTKLGAAKLANAAALGTKVDITHMAVGDGGGKLPDPDVNQ...,Photorhabdus australis,Phage tail fiber repeat protein,375,No
4046,WP_065786462.1,MSGTTDFVGVRVFSDLRSTVAKIDTRDSTVIGLVLPAPLADNAAFP...,Ensifer,MULTISPECIES: phage tail sheath family protein,418,No
24060,WP_010400143.1,MAFDMDAVREVFRGELKGMLDERDKKSNDLHEQIEHLKTKSEADAA...,Paracoccus sp. TRP,phage major capsid protein,399,No
31397,EJX10220.1,MPPAYVYDDTATNYKSAENASVAFLSQSLGPLLRKIENELERKLIP...,gut metagenome,Phage portal protein,129,No
18222,SET02381.1,MPENKVVFGLKNAHYAVITEDETTGELTYGTPKPLPGSVELSLEGR...,Salinibacillus kushneri,"phage major tail protein, phi13 family",192,No
...,...,...,...,...,...,...
50137,WP_062067772.1,MKTNDLIEQRAGIVARMNSAHEADDNTAFEAAETELRGLDAKLERQ...,Sphingobium baderi,phage major capsid protein,401,No
23427,SLM15270.1,MDSDVKEMLDNLGKEWKAFRDVNDQRLAAIEAKQGHAELDAKLAAI...,uncultured spirochete,"Phage major capsid protein, HK97 family",393,No
54986,QBP32413.1,MIKPQEEVDWTKPEEHFAWALRNMPTFAGTGAVTHPGFLTQWSKHL...,Mycobacterium phage AvatarAhPeg,minor tail protein,148,No
41700,WP_118737456.1,MRLFDTLRHSWDVFRNREPTYLNRGFASSYRPDRTQLSRGNERSIV...,Firmicutes bacterium AM31-12AC,phage portal protein,430,No


In [15]:
# We will obtain the features through the random sample as running this code on the full dataframe is time consuming
descriptors_df = ProteinDescritors(dataset=df_negative_cdhit_randomsampled , col='Sequence')    # Descriptors obtained from random sample
df_negative_cdhit_descriptors = descriptors_df.get_all_physicochemical(n_jobs = -1)             # Obtain physicochemical features of our sequences
# df_negative_cdhit_descriptors = descriptors_df.get_all(n_jobs = -1)  # Obtain all features of our sequences. Note: When running this code and then performing ML, the results of each model hinted towards Overfitting behaviour!
df_negative_cdhit_descriptors

,ID,Sequence,Organism,Structure,Size,Protein Acr,length,charge,chargedensity,formulaC,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,OCQ54497.1,MKYFAILTKLGAAKLANAAALGTKVDITHMAVGDGGGKLPDPDVNQ...,Photorhabdus australis,Phage tail fiber repeat protein,375,No,375.0,0.185,0.000005,1795,...,6.861796,24.517600,0.325333,0.325333,0.349333,55920,56045,87.200000,1.583707,0.354667
1,WP_065786462.1,MSGTTDFVGVRVFSDLRSTVAKIDTRDSTVIGLVLPAPLADNAAFP...,Ensifer,MULTISPECIES: phage tail sheath family protein,418,No,418.0,-4.783,-0.000107,1983,...,5.540347,27.405981,0.313397,0.282297,0.404306,48930,49055,100.406699,1.221124,0.421053
2,WP_010400143.1,MAFDMDAVREVFRGELKGMLDERDKKSNDLHEQIEHLKTKSEADAA...,Paracoccus sp. TRP,phage major capsid protein,399,No,399.0,-12.463,-0.000288,1896,...,5.054601,31.139850,0.368421,0.280702,0.345865,26930,26930,90.025063,1.567293,0.393484
3,EJX10220.1,MPPAYVYDDTATNYKSAENASVAFLSQSLGPLLRKIENELERKLIP...,gut metagenome,Phage portal protein,129,No,129.0,1.007,0.000069,642,...,7.964634,50.803876,0.286822,0.341085,0.325581,14440,14440,76.434109,2.753101,0.317829
4,SET02381.1,MPENKVVFGLKNAHYAVITEDETTGELTYGTPKPLPGSVELSLEGR...,Salinibacillus kushneri,"phage major tail protein, phi13 family",192,No,192.0,-14.850,-0.000703,922,...,4.476889,35.735469,0.338542,0.307292,0.359375,20400,20400,64.010417,1.942448,0.296875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,WP_062067772.1,MKTNDLIEQRAGIVARMNSAHEADDNTAFEAAETELRGLDAKLERQ...,Sphingobium baderi,phage major capsid protein,401,No,401.0,-14.370,-0.000335,1863,...,4.910912,32.287531,0.329177,0.301746,0.344140,23950,23950,88.703242,1.646185,0.401496
1118,SLM15270.1,MDSDVKEMLDNLGKEWKAFRDVNDQRLAAIEAKQGHAELDAKLAAI...,uncultured spirochete,"Phage major capsid protein, HK97 family",393,No,393.0,-5.847,-0.000137,1885,...,5.280195,24.271781,0.356234,0.295165,0.338422,66350,66350,83.027990,1.350254,0.384224
1119,QBP32413.1,MIKPQEEVDWTKPEEHFAWALRNMPTFAGTGAVTHPGFLTQWSKHL...,Mycobacterium phage AvatarAhPeg,minor tail protein,148,No,148.0,-5.560,-0.000323,762,...,5.339023,50.092568,0.304054,0.277027,0.317568,37470,37470,66.621622,2.074730,0.317568
1120,WP_118737456.1,MRLFDTLRHSWDVFRNREPTYLNRGFASSYRPDRTQLSRGNERSIV...,Firmicutes bacterium AM31-12AC,phage portal protein,430,No,430.0,3.391,0.000069,2133,...,8.566575,37.554419,0.311628,0.290698,0.360465,30370,30495,88.209302,2.112465,0.355814


#### Removal of non-numeric columns and conversion of 'ID' column into dataframe index

- Remove columns: 'Sequence', 'Organism', 'Family', 'Type'
- Convert 'Protein Acr' column to numerical (Yes = 1; No = 0)
- Turn 'ID' column into index

In [16]:
df_positive_cdhit_descriptors = df_positive_cdhit_descriptors.drop(columns=['Sequence', 'Organism', 'Family', 'Type', 'Size']) # Drops non-numerical columns and 'Size' column (we have 'length' column)
df_positive_cdhit_descriptors

,ID,Protein Acr,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,RGB60049.1,Yes,105.0,-2.901,-0.000240,539,826,136,171,3,...,4.980767,34.719048,0.266667,0.323810,0.390476,14440,14440,82.666667,1.931333,0.361905
1,WP_103240931.1,Yes,195.0,-17.844,-0.000784,1004,1492,274,310,11,...,4.608869,34.520000,0.343590,0.282051,0.307692,50420,50670,67.538462,2.295333,0.343590
2,WP_086652143.1,Yes,179.0,-20.646,-0.001021,888,1361,223,299,7,...,4.386572,54.459832,0.402235,0.217877,0.346369,25900,25900,81.899441,1.828212,0.391061
3,WP_012242545.1,Yes,144.0,-0.834,-0.000050,757,1149,189,232,4,...,6.289712,33.052083,0.291667,0.291667,0.402778,41940,42065,82.430556,1.778750,0.319444
4,WP_062681378.1,Yes,150.0,-19.684,-0.001198,721,1113,185,244,3,...,4.110619,34.327400,0.340000,0.286667,0.373333,11460,11460,98.333333,1.572533,0.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,CNL87740.1,Yes,146.0,-5.762,-0.000366,693,1088,180,219,6,...,4.923246,27.177397,0.301370,0.294521,0.369863,19480,19480,94.794521,0.922123,0.397260
1118,WP_050090803.1,Yes,162.0,-19.388,-0.001103,772,1178,204,259,3,...,4.306713,39.247531,0.327160,0.333333,0.327160,11460,11460,88.024691,1.628951,0.407407
1119,WP_071704514.1,Yes,88.0,0.198,0.000021,414,624,110,128,2,...,6.711741,27.969318,0.306818,0.340909,0.295455,19480,19480,72.159091,0.951818,0.363636
1120,CFQ72446.1,Yes,141.0,-15.925,-0.001013,681,1026,178,234,7,...,4.096693,41.008511,0.283688,0.361702,0.333333,23950,24075,78.226950,1.891064,0.375887


In [17]:
df_negative_cdhit_descriptors = df_negative_cdhit_descriptors.drop(columns=['Sequence', 'Organism', 'Structure', 'Size']) # Drops non-numerical columns and 'Size' column (we have 'length' column)
df_negative_cdhit_descriptors

,ID,Protein Acr,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,OCQ54497.1,No,375.0,0.185,0.000005,1795,2840,492,567,6,...,6.861796,24.517600,0.325333,0.325333,0.349333,55920,56045,87.200000,1.583707,0.354667
1,WP_065786462.1,No,418.0,-4.783,-0.000107,1983,3170,540,616,8,...,5.540347,27.405981,0.313397,0.282297,0.404306,48930,49055,100.406699,1.221124,0.421053
2,WP_010400143.1,No,399.0,-12.463,-0.000288,1896,3032,510,613,12,...,5.054601,31.139850,0.368421,0.280702,0.345865,26930,26930,90.025063,1.567293,0.393484
3,EJX10220.1,No,129.0,1.007,0.000069,642,1009,187,205,1,...,7.964634,50.803876,0.286822,0.341085,0.325581,14440,14440,76.434109,2.753101,0.317829
4,SET02381.1,No,192.0,-14.850,-0.000703,922,1412,232,323,4,...,4.476889,35.735469,0.338542,0.307292,0.359375,20400,20400,64.010417,1.942448,0.296875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,WP_062067772.1,No,401.0,-14.370,-0.000335,1863,2959,523,613,7,...,4.910912,32.287531,0.329177,0.301746,0.344140,23950,23950,88.703242,1.646185,0.401496
1118,SLM15270.1,No,393.0,-5.847,-0.000137,1885,2921,495,596,10,...,5.280195,24.271781,0.356234,0.295165,0.338422,66350,66350,83.027990,1.350254,0.384224
1119,QBP32413.1,No,148.0,-5.560,-0.000323,762,1129,203,236,5,...,5.339023,50.092568,0.304054,0.277027,0.317568,37470,37470,66.621622,2.074730,0.317568
1120,WP_118737456.1,No,430.0,3.391,0.000069,2133,3424,588,679,18,...,8.566575,37.554419,0.311628,0.290698,0.360465,30370,30495,88.209302,2.112465,0.355814


### Joining the Positive and Negative Datasets together
Here we will join dataframes ***'df_positive_cdhit_descriptors'*** with ***'df_negative_cdhit_descriptors'***.

What will help us distinguish the origin of the rows will be our 'Protein Acr' column: 
- '1' integer value = 'Yes' which means this row came from the POSITIVE dataset;
- '0' integer value = 'No' which means this row came from the NEGATIVE dataset.

In [18]:
df_cdhit_final = pd.concat([df_positive_cdhit_descriptors, df_negative_cdhit_descriptors])      # Concatenates the positive and negative dataframes
df_cdhit_final['Protein Acr'] = LabelEncoder().fit_transform(df_cdhit_final['Protein Acr'])     # Replace 'Yes' & 'No' values in 'Protein Acr' with numericals using sklearn's LabelEncoder
df_cdhit_final = df_cdhit_final.rename(columns={'Protein Acr': 'Protein_Acr'})                  # Renaming 'Protein Acr' column to avoid future formatting or column read errors
df_cdhit_final.reset_index(drop=True, inplace=True)                                             # Resets index
df_cdhit_final.set_index('ID', inplace=True)                                                    # Set the "ID" column as the index
df_cdhit_final

# ## If the Propythia function used was get_all(), comment the above code and uncomment this code:
# df_positive_cdhit_descriptors = df_positive_cdhit_descriptors.rename(columns={'Protein Acr': 'Protein_Acr', 'ID_x': 'ID'})  # Rename 'Protein Acr' to 'Protein_Acr' and 'ID_x' to 'ID'
# df_negative_cdhit_descriptors = df_negative_cdhit_descriptors.rename(columns={'Protein Acr': 'Protein_Acr', 'ID_x': 'ID'})  # Rename 'Protein Acr' to 'Protein_Acr' and 'ID_x' to 'ID'
# df_cdhit_final = pd.concat([df_positive_cdhit_descriptors, df_negative_cdhit_descriptors])                                  # Concatenates the positive and negative dataframes
# df_cdhit_final['Protein_Acr'] = LabelEncoder().fit_transform(df_cdhit_final['Protein_Acr'])                                 # Replace 'Yes' & 'No' values in 'Protein Acr' with numericals using sklearn's LabelEncoder
# df_cdhit_final.reset_index(drop=True, inplace=True)                                                                         # Resets index
# df_cdhit_final.set_index('ID', inplace=True)                                                                                # Set the "ID" column as the index
# df_cdhit_final

,Protein_Acr,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,tot,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
ID,,,,,,,,,,,,,,,,,,,,,
RGB60049.1,1,105.0,-2.901,-0.000240,539,826,136,171,3,1912,...,4.980767,34.719048,0.266667,0.323810,0.390476,14440,14440,82.666667,1.931333,0.361905
WP_103240931.1,1,195.0,-17.844,-0.000784,1004,1492,274,310,11,3583,...,4.608869,34.520000,0.343590,0.282051,0.307692,50420,50670,67.538462,2.295333,0.343590
WP_086652143.1,1,179.0,-20.646,-0.001021,888,1361,223,299,7,3187,...,4.386572,54.459832,0.402235,0.217877,0.346369,25900,25900,81.899441,1.828212,0.391061
WP_012242545.1,1,144.0,-0.834,-0.000050,757,1149,189,232,4,2698,...,6.289712,33.052083,0.291667,0.291667,0.402778,41940,42065,82.430556,1.778750,0.319444
WP_062681378.1,1,150.0,-19.684,-0.001198,721,1113,185,244,3,2598,...,4.110619,34.327400,0.340000,0.286667,0.373333,11460,11460,98.333333,1.572533,0.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WP_062067772.1,0,401.0,-14.370,-0.000335,1863,2959,523,613,7,6868,...,4.910912,32.287531,0.329177,0.301746,0.344140,23950,23950,88.703242,1.646185,0.401496
SLM15270.1,0,393.0,-5.847,-0.000137,1885,2921,495,596,10,6830,...,5.280195,24.271781,0.356234,0.295165,0.338422,66350,66350,83.027990,1.350254,0.384224
QBP32413.1,0,148.0,-5.560,-0.000323,762,1129,203,236,5,2728,...,5.339023,50.092568,0.304054,0.277027,0.317568,37470,37470,66.621622,2.074730,0.317568


In [19]:
# Creation of the .csv file containing the dataset of our CD-HIT results
df_cdhit_final.to_csv('PhageAcr_ML_dataset_cdhit.csv', index=True)          # Creates a .csv file

## - MMSeqs2 Dataframes
- Application of Propythia on both positive and negative MMSeqs2 results;
- Random selection of rows from negative dataset;
- Creation of MMSeqs2 dataframe.

### df_positive_mmseqs

In [20]:
read_seqs = ReadSequence()
res = read_seqs.par_preprocessing(dataset=df_positive_mmseqs, col='Sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
res

,ID,Sequence,Organism,Family,Type,Size,Protein Acr
0,RGB60049.1,MSIYTDMIPAILLVNDPQDSLSGAPIENYVKVSNINVAIYKNDSFK...,Absiella sp.,AcrIIA8,II-A,105,Yes
1,WP_103240931.1,MSCPFQAMEGGNGMERKMALREFCGRYRKGDFKGTERAVQIEAGWY...,Acetatifactor muris,AcrIIA11,II-A,195,Yes
2,WP_086652143.1,MELIHTSDEVIKKIHKDGTFDTFLFFSASKYLAGSVASRKHYTYKI...,Acetobacter cibinongensis,AcrIF11,I-F,179,Yes
3,WP_012242545.1,MEKQQLLKDLIQAFNSGSFESSDVKVQIKAGWYDWFCKDSSLKNKT...,Acholeplasma laidlawii,AcrIIA11,II-A,144,Yes
4,WP_062681378.1,MQLFHTSPSEISTITSTGRFGSFLFFSARAYTMTAGEALVYSLEID...,Achromobacter denitrificans,AcrIF11,I-F,150,Yes
...,...,...,...,...,...,...,...
1101,WP_011192267.1,MNFGQALQALKAGYKVARIGWNGKGMFLILISGTKDVEPCEGTPYA...,Yersinia pseudotuberculosis,AcrIIA7,II-A,146,Yes
1102,AIN13836.1,MDSASSHGCGFVYTYEINDEKIASSRDLDNRTEEVYAFLRAELNID...,Yersinia pseudotuberculosis,AcrIF11,I-F,124,Yes
1103,WP_071704514.1,MNFGEALEAVKSGKKIARSGWNGAAQFVIKAGGYTVSEARPGSDYA...,Yersinia ruckeri,AcrIIA7,II-A,88,Yes
1104,CFQ72446.1,MKLFHGSYSSTAPVMQIGQFTQVNGSENLYDGIFASDSMDSASSHG...,Yersinia similis,AcrIF11,I-F,141,Yes


In [21]:
descriptors_df = ProteinDescritors(dataset=df_positive_mmseqs , col='Sequence')
df_positive_mmseqs_descriptors = descriptors_df.get_all_physicochemical(n_jobs = -1)    # Obtain physicochemical features of our sequences
# df_positive_mmseqs_descriptors = descriptors_df.get_all(n_jobs = -1)  # Obtain all features of our sequences. Note: When running this code and then performing ML, the results of each model hinted towards Overfitting behaviour!
df_positive_mmseqs_descriptors

,ID,Sequence,Organism,Family,Type,Size,Protein Acr,length,charge,chargedensity,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,RGB60049.1,MSIYTDMIPAILLVNDPQDSLSGAPIENYVKVSNINVAIYKNDSFK...,Absiella sp.,AcrIIA8,II-A,105,Yes,105.0,-2.901,-0.000240,...,4.980767,34.719048,0.266667,0.323810,0.390476,14440,14440,82.666667,1.931333,0.361905
1,WP_103240931.1,MSCPFQAMEGGNGMERKMALREFCGRYRKGDFKGTERAVQIEAGWY...,Acetatifactor muris,AcrIIA11,II-A,195,Yes,195.0,-17.844,-0.000784,...,4.608869,34.520000,0.343590,0.282051,0.307692,50420,50670,67.538462,2.295333,0.343590
2,WP_086652143.1,MELIHTSDEVIKKIHKDGTFDTFLFFSASKYLAGSVASRKHYTYKI...,Acetobacter cibinongensis,AcrIF11,I-F,179,Yes,179.0,-20.646,-0.001021,...,4.386572,54.459832,0.402235,0.217877,0.346369,25900,25900,81.899441,1.828212,0.391061
3,WP_012242545.1,MEKQQLLKDLIQAFNSGSFESSDVKVQIKAGWYDWFCKDSSLKNKT...,Acholeplasma laidlawii,AcrIIA11,II-A,144,Yes,144.0,-0.834,-0.000050,...,6.289712,33.052083,0.291667,0.291667,0.402778,41940,42065,82.430556,1.778750,0.319444
4,WP_062681378.1,MQLFHTSPSEISTITSTGRFGSFLFFSARAYTMTAGEALVYSLEID...,Achromobacter denitrificans,AcrIF11,I-F,150,Yes,150.0,-19.684,-0.001198,...,4.110619,34.327400,0.340000,0.286667,0.373333,11460,11460,98.333333,1.572533,0.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,WP_011192267.1,MNFGQALQALKAGYKVARIGWNGKGMFLILISGTKDVEPCEGTPYA...,Yersinia pseudotuberculosis,AcrIIA7,II-A,146,Yes,146.0,-5.861,-0.000371,...,4.787514,26.293151,0.315068,0.287671,0.376712,20970,20970,98.150685,0.857192,0.410959
1102,AIN13836.1,MDSASSHGCGFVYTYEINDEKIASSRDLDNRTEEVYAFLRAELNID...,Yersinia pseudotuberculosis,AcrIF11,I-F,124,Yes,124.0,-16.823,-0.001221,...,4.187181,39.440323,0.322581,0.338710,0.298387,16960,17085,79.596774,2.216129,0.387097
1103,WP_071704514.1,MNFGEALEAVKSGKKIARSGWNGAAQFVIKAGGYTVSEARPGSDYA...,Yersinia ruckeri,AcrIIA7,II-A,88,Yes,88.0,0.198,0.000021,...,6.711741,27.969318,0.306818,0.340909,0.295455,19480,19480,72.159091,0.951818,0.363636
1104,CFQ72446.1,MKLFHGSYSSTAPVMQIGQFTQVNGSENLYDGIFASDSMDSASSHG...,Yersinia similis,AcrIF11,I-F,141,Yes,141.0,-15.925,-0.001013,...,4.096693,41.008511,0.283688,0.361702,0.333333,23950,24075,78.226950,1.891064,0.375887


### df_negative_mmseqs

In [22]:
read_seqs = ReadSequence()
res = read_seqs.par_preprocessing(dataset=df_negative_mmseqs, col='Sequence', B ='N', Z = 'Q', U = 'C', O = 'K', J = 'I', X = '')
res

,ID,Sequence,Organism,Structure,Size,Protein Acr
0,WP_132722810.1,MTNNTDTFLHYREGKTQSQRFLKNLDPENLKLNDLDVADWLLFAFN...,Tenacibaculum sp. M341,phage baseplate protein,1042,No
1,WP_123214754.1,MADCNDILDILKRDGTGQEQRYTEALRTDYFRLQDLDIADWMLFAY...,Sinomicrobium pectinilyticum,phage baseplate protein,1051,No
2,ARW58754.1,MGIYVKLINVPVSERIYCDLNAWIQLEPRDNVQNMDSIVQKILMIV...,Erwinia phage vB_EamM_Y3,baseplate protein,139,No
3,WP_096978906.1,MERDLYTPTGAQTSEAQLLEYTFEMLLSGCFFIELAKVVAVRGKAP...,Escherichia coli,phage baseplate protein,230,No
4,WP_050336321.1,MNNPYSQSQNQSNDSDAFASSFNKLLNSNYFIRLATVTAVRGTAPN...,Yersinia enterocolitica,phage baseplate protein,236,No
...,...,...,...,...,...,...
58281,WP_011840497.1,MDKEIAAILERVVKDVARIDNELSKKAEAAFAEVKNMGNLSTETKA...,Rhodobacter sphaeroides,phage major capsid protein,403,No
58282,AXC34520.1,MKTRNKLTEALLGRGGFAELFAPKPTNKEEIKNEEKLLKADQMVEM...,Vibrio phage YC,portal protein,566,No
58283,YP_009222143.1,MATWKSRRKQKHEAKQKALLEKASSEAPAPRLRLGEIGTTGLAAIQ...,Vibrio phage qdvp001,portal protein,554,No
58284,WP_057903243.1,MVLEVNAPLLDDRFIEYARISNNGTFFYPAELEISTGDLLNLVDYH...,Lactobacillus bifermentans,phage portal protein,499,No


##### Random selection of n rows (negative) = n rows (positive)

In [23]:
# Random selection of n rows through pandas sample() method (where n = number of rows in the positive counterpart)
# Note: Paramater 'random_state' of the sample() method ensures reproducibility of the random selection
df_negative_mmseqs_randomsample = df_negative_mmseqs.sample(n=df_positive_mmseqs_descriptors.shape[0], random_state=1)
df_negative_mmseqs_randomsample

,ID,Sequence,Organism,Structure,Size,Protein Acr
8393,WP_074952311.1,MATGLIIPGVQVTVVKEVLPQQLAPSGVLGITGFLESAQGKVIRAS...,Myxococcus fulvus,phage tail sheath protein,456,No
20417,BAR33603.1,MTKADRFSQTKERLDNTYSWRSEEGYDAKWHRMIDLYRGKTFGGTG...,uncultured Mediterranean phage uvMED,Head-tail connector protein,631,No
4164,YP_006489127.1,MALLSPGIELKENTTQSTVVQNATGRAAMAGKFQWGPAFQVIQVTN...,Acinetobacter phage ZZ1,tail sheath protein,659,No
32260,OJI93368.1,MMLIEEASVPDEVLPLAAFKAHLRLGTGFAQDDVQDPVLKSFLKAA...,Planktotalea frisia,phage gp6-like head-tail connector protein,199,No
24664,WP_003715197.1,MGINELNDAWISAGQKVDDMNAKLNAAVLDDAFDKDAFKSMKAQRD...,Lactobacillus oris,phage major capsid protein,400,No
...,...,...,...,...,...,...
655,AYD80647.1,MSDLLKQHFRATNGLDAGGNKVINVALADRNVKTDGVSVEYVIQEN...,Klebsiella phage KP179,long tail fiber proximal subunit,1281,No
25543,WP_082645163.1,MTRRQGPETARDGRKTGYLAADWVTEGSLIPVKQGMVGTALLNRFK...,Phaeobacter sp. CECT 7735,phage major capsid protein,261,No
16022,WP_077534944.1,MNLQTAQDTDNGYSVFEQSLLRYIAAGLGVSYEQLSRNYAQMSYST...,Escherichia coli,phage portal protein,195,No
36847,GAR05010.1,MSGDSVVISSVAYPDPRKLALVADMQYHEPYTSAALNRKLRGILRE...,Salmonella enterica,phage Tail Collar Domain protein,727,No


In [24]:
# We will obtain the features through the random sample as running this code on the full dataframe is time consuming
descriptors_df = ProteinDescritors(dataset=df_negative_mmseqs_randomsample , col='Sequence')    # Descriptors obtained from random sample
df_negative_mmseqs_descriptors = descriptors_df.get_all_physicochemical(n_jobs = -1)            # Obtain physicochemical features of our sequences
# df_negative_mmseqs_descriptors = descriptors_df.get_all(n_jobs = -1)  # Obtain all features of our sequences. Note: When running this code and then performing ML, the results of each model hinted towards Overfitting behaviour!
df_negative_mmseqs_descriptors

,ID,Sequence,Organism,Structure,Size,Protein Acr,length,charge,chargedensity,formulaC,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,WP_074952311.1,MATGLIIPGVQVTVVKEVLPQQLAPSGVLGITGFLESAQGKVIRAS...,Myxococcus fulvus,phage tail sheath protein,456,No,456.0,0.450,0.000009,2111,...,6.988036,25.222588,0.293860,0.302632,0.388158,31400,31400,97.894737,1.169057,0.403509
1,BAR33603.1,MTKADRFSQTKERLDNTYSWRSEEGYDAKWHRMIDLYRGKTFGGTG...,uncultured Mediterranean phage uvMED,Head-tail connector protein,631,No,631.0,-38.977,-0.000545,3107,...,4.677190,46.739176,0.286846,0.301109,0.323296,77240,77365,70.586371,2.137179,0.347068
2,YP_006489127.1,MALLSPGIELKENTTQSTVVQNATGRAAMAGKFQWGPAFQVIQVTN...,Acinetobacter phage ZZ1,tail sheath protein,659,No,659.0,-17.006,-0.000238,3158,...,4.885277,32.861927,0.289833,0.312595,0.371775,73230,73480,87.238240,1.361548,0.399090
3,OJI93368.1,MMLIEEASVPDEVLPLAAFKAHLRLGTGFAQDDVQDPVLKSFLKAA...,Planktotalea frisia,phage gp6-like head-tail connector protein,199,No,199.0,-3.992,-0.000184,972,...,5.254219,44.842211,0.351759,0.276382,0.336683,23950,24075,91.356784,1.228794,0.447236
4,WP_003715197.1,MGINELNDAWISAGQKVDDMNAKLNAAVLDDAFDKDAFKSMKAQRD...,Lactobacillus oris,phage major capsid protein,400,No,400.0,-12.579,-0.000287,1902,...,4.891132,10.562500,0.332500,0.312500,0.342500,28420,28420,86.375000,1.830825,0.395000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,AYD80647.1,MSDLLKQHFRATNGLDAGGNKVINVALADRNVKTDGVSVEYVIQEN...,Klebsiella phage KP179,long tail fiber proximal subunit,1281,No,1281.0,3.049,0.000022,6044,...,8.069395,24.002966,0.252147,0.313817,0.398907,168110,168110,80.257611,1.705667,0.330211
1102,WP_082645163.1,MTRRQGPETARDGRKTGYLAADWVTEGSLIPVKQGMVGTALLNRFK...,Phaeobacter sp. CECT 7735,phage major capsid protein,261,No,261.0,-9.853,-0.000355,1223,...,4.653715,25.518391,0.325670,0.252874,0.406130,26470,26470,98.007663,0.908123,0.448276
1103,WP_077534944.1,MNLQTAQDTDNGYSVFEQSLLRYIAAGLGVSYEQLSRNYAQMSYST...,Escherichia coli,phage portal protein,195,No,195.0,-1.186,-0.000054,944,...,5.727518,60.915897,0.374359,0.235897,0.287179,39420,39545,65.743590,2.140821,0.379487
1104,GAR05010.1,MSGDSVVISSVAYPDPRKLALVADMQYHEPYTSAALNRKLRGILRE...,Salmonella enterica,phage Tail Collar Domain protein,727,No,727.0,-16.053,-0.000210,3331,...,5.210737,30.356410,0.283356,0.357634,0.354883,72310,72560,83.947730,1.375736,0.349381


#### Removal of non-numeric columns and conversion of 'ID' column into dataframe index

- Remove columns: 'Sequence', 'Organism', 'Family', 'Type'
- Convert 'Protein Acr' column to numerical (Yes = 1; No = 0)
- Turn 'ID' column into index

In [25]:
df_positive_mmseqs_descriptors = df_positive_mmseqs_descriptors.drop(columns=['Sequence', 'Organism', 'Family', 'Type', 'Size']) # Drops non-numerical columns and 'Size' column (we have 'length' column)
df_positive_mmseqs_descriptors

,ID,Protein Acr,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,RGB60049.1,Yes,105.0,-2.901,-0.000240,539,826,136,171,3,...,4.980767,34.719048,0.266667,0.323810,0.390476,14440,14440,82.666667,1.931333,0.361905
1,WP_103240931.1,Yes,195.0,-17.844,-0.000784,1004,1492,274,310,11,...,4.608869,34.520000,0.343590,0.282051,0.307692,50420,50670,67.538462,2.295333,0.343590
2,WP_086652143.1,Yes,179.0,-20.646,-0.001021,888,1361,223,299,7,...,4.386572,54.459832,0.402235,0.217877,0.346369,25900,25900,81.899441,1.828212,0.391061
3,WP_012242545.1,Yes,144.0,-0.834,-0.000050,757,1149,189,232,4,...,6.289712,33.052083,0.291667,0.291667,0.402778,41940,42065,82.430556,1.778750,0.319444
4,WP_062681378.1,Yes,150.0,-19.684,-0.001198,721,1113,185,244,3,...,4.110619,34.327400,0.340000,0.286667,0.373333,11460,11460,98.333333,1.572533,0.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,WP_011192267.1,Yes,146.0,-5.861,-0.000371,697,1094,178,220,6,...,4.787514,26.293151,0.315068,0.287671,0.376712,20970,20970,98.150685,0.857192,0.410959
1102,AIN13836.1,Yes,124.0,-16.823,-0.001221,594,904,164,204,5,...,4.187181,39.440323,0.322581,0.338710,0.298387,16960,17085,79.596774,2.216129,0.387097
1103,WP_071704514.1,Yes,88.0,0.198,0.000021,414,624,110,128,2,...,6.711741,27.969318,0.306818,0.340909,0.295455,19480,19480,72.159091,0.951818,0.363636
1104,CFQ72446.1,Yes,141.0,-15.925,-0.001013,681,1026,178,234,7,...,4.096693,41.008511,0.283688,0.361702,0.333333,23950,24075,78.226950,1.891064,0.375887


In [26]:
df_negative_mmseqs_descriptors = df_negative_mmseqs_descriptors.drop(columns=['Sequence', 'Organism', 'Structure', 'Size']) # Drops non-numerical columns and 'Size' column (we have 'length' column)
df_negative_mmseqs_descriptors

,ID,Protein Acr,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
0,WP_074952311.1,No,456.0,0.450,0.000009,2111,3396,576,672,6,...,6.988036,25.222588,0.293860,0.302632,0.388158,31400,31400,97.894737,1.169057,0.403509
1,BAR33603.1,No,631.0,-38.977,-0.000545,3107,4742,838,1017,30,...,4.677190,46.739176,0.286846,0.301109,0.323296,77240,77365,70.586371,2.137179,0.347068
2,YP_006489127.1,No,659.0,-17.006,-0.000238,3158,4900,828,1009,17,...,4.885277,32.861927,0.289833,0.312595,0.371775,73230,73480,87.238240,1.361548,0.399090
3,OJI93368.1,No,199.0,-3.992,-0.000184,972,1509,257,288,8,...,5.254219,44.842211,0.351759,0.276382,0.336683,23950,24075,91.356784,1.228794,0.447236
4,WP_003715197.1,No,400.0,-12.579,-0.000287,1902,3013,515,632,11,...,4.891132,10.562500,0.332500,0.312500,0.342500,28420,28420,86.375000,1.830825,0.395000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,AYD80647.1,No,1281.0,3.049,0.000022,6044,9544,1658,1995,16,...,8.069395,24.002966,0.252147,0.313817,0.398907,168110,168110,80.257611,1.705667,0.330211
1102,WP_082645163.1,No,261.0,-9.853,-0.000355,1223,1946,316,390,10,...,4.653715,25.518391,0.325670,0.252874,0.406130,26470,26470,98.007663,0.908123,0.448276
1103,WP_077534944.1,No,195.0,-1.186,-0.000054,944,1449,263,311,10,...,5.727518,60.915897,0.374359,0.235897,0.287179,39420,39545,65.743590,2.140821,0.379487
1104,GAR05010.1,No,727.0,-16.053,-0.000210,3331,5265,903,1113,15,...,5.210737,30.356410,0.283356,0.357634,0.354883,72310,72560,83.947730,1.375736,0.349381


### Joining the Positive and Negative Datasets together
Here we will join dataframes ***'df_positive_mmseqs_descriptors'*** with ***'df_negative_mmseqs_descriptors'***.

What will help us distinguish the origin of the rows will be our 'Protein Acr' column: 
- '1' integer value = 'Yes' which means this row came from the POSITIVE dataset;
- '0' integer value = 'No' which means this row came from the NEGATIVE dataset.

In [27]:
df_mmseqs_final = pd.concat([df_positive_mmseqs_descriptors, df_negative_mmseqs_descriptors])  # Concatenates the positive and negative dataframes
df_mmseqs_final['Protein Acr'] = LabelEncoder().fit_transform(df_mmseqs_final['Protein Acr'])  # Replace 'Yes' & 'No' values in 'Protein Acr' with numericals using sklearn's LabelEncoder
df_mmseqs_final = df_mmseqs_final.rename(columns={'Protein Acr': 'Protein_Acr'})               # Renaming 'Protein Acr' column to avoid future formatting or column read errors
df_mmseqs_final.reset_index(drop=True, inplace=True)                                           # Resets index
df_mmseqs_final.set_index('ID', inplace=True)                                                   # Set the "ID" column as the index
df_mmseqs_final

# ## If the Propythia function used was get_all(), comment the above code and uncomment this code:
# df_positive_mmseqs_descriptors = df_positive_mmseqs_descriptors.rename(columns={'Protein Acr': 'Protein_Acr', 'ID_x': 'ID'})    # Rename 'Protein Acr' to 'Protein_Acr' and 'ID_x' to 'ID'
# df_negative_mmseqs_descriptors = df_negative_mmseqs_descriptors.rename(columns={'Protein Acr': 'Protein_Acr', 'ID_x': 'ID'})    # Rename 'Protein Acr' to 'Protein_Acr' and 'ID_x' to 'ID'
# df_mmseqs_final = pd.concat([df_positive_mmseqs_descriptors, df_negative_mmseqs_descriptors])                                   # Concatenates the positive and negative dataframes
# df_mmseqs_final['Protein_Acr'] = LabelEncoder().fit_transform(df_mmseqs_final['Protein_Acr'])                                   # Replace 'Yes' & 'No' values in 'Protein Acr' with numericals using sklearn's LabelEncoder
# df_mmseqs_final.reset_index(drop=True, inplace=True)                                                                            # Resets index
# df_mmseqs_final.set_index('ID', inplace=True)                                                                                   # Set the "ID" column as the index
# df_mmseqs_final

,Protein_Acr,length,charge,chargedensity,formulaC,formulaH,formulaN,formulaO,formulaS,tot,...,IsoelectricPoint,Instability_index,SecStruct_helix,SecStruct_turn,SecStruct_sheet,Molar_extinction_coefficient_reduced,Molar_extinction_coefficient_oxidized,aliphatic_index,bomanindex,hydrophobic_ratio
ID,,,,,,,,,,,,,,,,,,,,,
RGB60049.1,1,105.0,-2.901,-0.000240,539,826,136,171,3,1912,...,4.980767,34.719048,0.266667,0.323810,0.390476,14440,14440,82.666667,1.931333,0.361905
WP_103240931.1,1,195.0,-17.844,-0.000784,1004,1492,274,310,11,3583,...,4.608869,34.520000,0.343590,0.282051,0.307692,50420,50670,67.538462,2.295333,0.343590
WP_086652143.1,1,179.0,-20.646,-0.001021,888,1361,223,299,7,3187,...,4.386572,54.459832,0.402235,0.217877,0.346369,25900,25900,81.899441,1.828212,0.391061
WP_012242545.1,1,144.0,-0.834,-0.000050,757,1149,189,232,4,2698,...,6.289712,33.052083,0.291667,0.291667,0.402778,41940,42065,82.430556,1.778750,0.319444
WP_062681378.1,1,150.0,-19.684,-0.001198,721,1113,185,244,3,2598,...,4.110619,34.327400,0.340000,0.286667,0.373333,11460,11460,98.333333,1.572533,0.426667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AYD80647.1,0,1281.0,3.049,0.000022,6044,9544,1658,1995,16,22198,...,8.069395,24.002966,0.252147,0.313817,0.398907,168110,168110,80.257611,1.705667,0.330211
WP_082645163.1,0,261.0,-9.853,-0.000355,1223,1946,316,390,10,4493,...,4.653715,25.518391,0.325670,0.252874,0.406130,26470,26470,98.007663,0.908123,0.448276
WP_077534944.1,0,195.0,-1.186,-0.000054,944,1449,263,311,10,3481,...,5.727518,60.915897,0.374359,0.235897,0.287179,39420,39545,65.743590,2.140821,0.379487


In [28]:
# Creation of the .csv file containing the dataset of our CD-HIT results
df_mmseqs_final.to_csv('PhageAcr_ML_dataset_mmseqs.csv', index=True)          # Creates a .csv file